In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 25.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pickle 
data = pickle.load(open('tfidf_corpus (1).pkl', 'rb'))

In [3]:
from tqdm.auto import tqdm
maximum_word = 0
for doc in tqdm(data['vectors']):
    max_index =max([v[0] for v in doc])
    maximum_word = max(maximum_word, max_index)
    

/home/amos/miniconda3/envs/GNNpapersearch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 2381173/2381173 [00:40<00:00, 59491.49it/s]


In [9]:
maximum_word

398742

In [4]:
import numpy as np
import pickle

ipca = pickle.load(open('ipca_0.pkl', 'rb'))

def generator(batch_size):
    for i in range(0, len(data['vectors'])//batch_size):
        x = data['vectors'][i*batch_size : (i+1)*batch_size]
        ids = data['ids'][i*batch_size : (i+1)*batch_size]
        embeddings = np.zeros((len(x), maximum_word+1))
        for i, doc in tqdm(enumerate(x)):
            for v in doc:
                embeddings[i, v[0]] = v[1]
                
        yield ids, embeddings


gen = generator(256)
tfidf_embeddings = []
for i, (ids, embeddings) in enumerate(gen):
    print(i, len(data['vectors'])//512)
    embds = ipca.transform(embeddings)
    tfidf_embeddings.append((ids, embds))
    

# one numpy array
tfidf_embeddings = np.concatenate([v[1] for v in tfidf_embeddings], axis=0)
tfidf_ids = np.concatenate([v[0] for v in tfidf_embeddings], axis=0)

pickle.dump({'ids':tfidf_ids, 'embeddings':tfidf_embeddings}, open('tfidf_embeddings.pkl', 'wb'))


256it [00:00, 1133.12it/s]0:00<?, ?it/s]


: 